# Advent of Code 2020, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202020%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL. Why? A language that doesn't affect the way you think about programming is not worth knowing.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [1]:
⍝ Helper functions and common settings
⎕FR ⎕PP ⎕IO←1287 34 0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
'segs'⎕CY'dfns'

⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

done

Rebuilding user command cache... done

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Report Repair
https://adventofcode.com/2020/day/1

Outer product sum, find locations of 2020 and multiply. 

In [2]:
⊢part1←×/d[⊃⍸2020=∘.+⍨d←⍎¨⊃⎕NGET'1'1]
assert 73371=part1

73371

In [3]:
⊢part2←×/d[⊃⍸2020=d∘.+d∘.+d]
assert 127642310=part2

127642310

### Day 2: Password Philosophy
https://adventofcode.com/2020/day/2

In [2]:
⎕IO←1
DAY02←{'- :'segs ⍵}¨⊃⎕NGET'data/2020/day02.txt'1

In [10]:
⊢result←+⌿↑{(b t l s)←⍵⋄(+/+/{(⍺=l)∧((⍎b)≥⍨≢⍵)∧((⍎t)≤⍨≢⍵)}⌸s)(⊃(l=(⍎b)⊃s)≠l=(⍎t)⊃s)}¨DAY02
assert 528 497≡result

┌→──────┐
│528 497│
└~──────┘